# 作業5：算出矩陣P、看法向量Q

## 有四資產 A、B、C、D
## A為6%
## B比D低2%
## C比A低3%

## 由上可知個資產收益率分別為
## A:6%
## B:D-2%
## C:3%
## D:未知
- - -
## ANS:
## 可得出的矩陣P為以下:
## A　B　C　D
## 1　 0　 0　0
## 0　-1　 0　1
## 1　 0　-1　0

## 看法向量q=[6%,2%,3%]^T

# 作業6：Black-Litterman

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 構造投資人個人觀點
# 建構資產選取矩陣P
pick1 = np.array([1,0,0,0])
q1 = np.array([0.06])
pick2 = np.array([0,-1,0,1])
q2 = np.array([-0.02])
pick3 = np.array([1,0,-1,0])
q3 = np.array([-0.03])
P = np.array([pick1,pick2,pick3])
Q = np.array([q1,q2,q3])

In [3]:
P

array([[ 1,  0,  0,  0],
       [ 0, -1,  0,  1],
       [ 1,  0, -1,  0]])

In [4]:
Q

array([[ 0.06],
       [-0.02],
       [-0.03]])

In [5]:
np.dot(Q.T,P)

array([[ 0.03,  0.02,  0.03, -0.02]])

In [6]:
def blacklitterman(returns,tau, P, Q):

    mu=returns.mean()

    sigma=returns.cov()

    pi1=mu

    ts = tau * sigma

    Omega = np.dot(np.dot(P,ts),P.T) * np.eye(Q.shape[0])

    middle = np.linalg.inv(np.dot(np.dot(P,ts),P.T) + Omega)  

    er = np.expand_dims(pi1,axis=0).T + np.dot(np.dot(np.dot(ts,P.T),middle),

                      (Q - np.expand_dims(np.dot(P,pi1.T),axis=1)))

    posteriorSigma = sigma + ts - np.dot(ts.dot(P.T).dot(middle).dot(P),ts)

    return [er, posteriorSigma]

In [7]:
Data_1432 = pd.read_csv('002/1432.csv', sep = ',')
Data_1432.index = pd.to_datetime(Data_1432.Date)
Data_1432.head()
DataRet_1432 = Data_1432.ROI
DataRet_1432.name = '1432_returns'

Data_1504 = pd.read_csv('002/1504.csv', sep = ',')
Data_1504.index = pd.to_datetime(Data_1504.Date)
Data_1504.head()
DataRet_1504 = Data_1504.ROI
DataRet_1504.name = '1504_returns'

Data_1707 = pd.read_csv('002/1707.csv', sep = ',')
Data_1707.index = pd.to_datetime(Data_1707.Date)
Data_1707.head()
DataRet_1707 = Data_1707.ROI
DataRet_1707.name = '1707_returns'

Data_2515 = pd.read_csv('002/2515.csv', sep = ',')
Data_2515.index = pd.to_datetime(Data_2515.Date)
Data_2515.head()
DataRet_2515 = Data_2515.ROI
DataRet_2515.name = '2515_returns'

In [8]:
retData = pd.concat([DataRet_1432,DataRet_1504,DataRet_1707,DataRet_2515],axis = 1).dropna()
# ROI數據單位是%，所以乘以0.01轉換成實際收益率
retData = retData.astype(np.float)*0.01
retData.head()

c:\users\a9025\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


,1432_returns,1504_returns,1707_returns,2515_returns
Date,,,,
2014-01-02,0.000000,0.000000,-0.013333,-0.004449
2014-01-03,-0.008889,-0.008785,-0.010135,-0.005587
2014-01-06,-0.002242,-0.008863,-0.030717,-0.023596
2014-01-07,0.002247,-0.013413,-0.003521,0.001151
2014-01-08,-0.006726,-0.003021,-0.007067,0.005747


In [9]:
# 修正後驗收益
res = blacklitterman(retData, 0.1, P, Q)

In [10]:
p_mean = pd.DataFrame(res[0],index = retData.columns,
                     columns = ['posterior_mean'])

In [11]:
p_mean

,posterior_mean
1432_returns,0.023684
1504_returns,0.011473
1707_returns,0.025674
2515_returns,0.001316


In [12]:
p_cov = res[1]
p_cov

,1432_returns,1504_returns,1707_returns,2515_returns
1432_returns,0.000382,0.000027,0.000095,0.000041
1504_returns,0.000027,0.000339,0.000081,0.000064
1707_returns,0.000095,0.000081,0.000489,0.000062
2515_returns,0.000041,0.000064,0.000062,0.000188


In [13]:
import portfolioopt as pfopt
target_ret2 = 0.0006 #設定目標收益率為0.0006
cov_mat2 = res[1]
avg_rets2 = pd.Series(res[0][:,0],index=retData.columns)
weights2 = pfopt.markowitz_portfolio(cov_mat2, avg_rets2, target_ret2, allow_short=False)
weights2

1432_returns    0.215026
1504_returns    0.214654
1707_returns    0.103153
2515_returns    0.467168
dtype: float64

# Markowitz模型配置完的資產配置為以上
### 東元(1504):0.214654
### 中工(2515):0.467168
### 太魯閣(1432):0.215026
### 葡萄王(1707):0.103153